<a href="https://colab.research.google.com/github/ImVis10/1PercentStrategy/blob/V1/1PercentStrategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import csv  
import re
import datetime

In [12]:
API_KEY = 'OTPBXZ24V7Q9A24S'

In [ ]:
def write_to_csv(stock,data, mode):


  # header = ['time', 'open', 'high', 'low', 'close', 'volume']

  with open(stock+'_minute.csv', mode, encoding='UTF8') as f:
      writer = csv.writer(f)

      # write the header
      # writer.writerow(header)

      # write the data
      writer.writerow(data)

In [ ]:
def read_csv():
  return None

In [ ]:
import csv
import requests

def get_stock_data(stock_name):
  time_periods ={
    # '1':'year1month1',
    # '2':'year1month2',
    # '3':'year1month3',
    # '4':'year1month4',
    # '5':'year1month5',
    # '6':'year1month6',
    # '7':'year1month7',
    # '8':'year1month8',
    # '9':'year1month9',
    # '10':'year1month10',
    # '11':'year1month11',
    # '12':'year1month12',
    # '13':'year2month1',
    # '14':'year2month2',
    # '15':'year2month3',
    # '16':'year2month4',
    # '17':'year2month5',
    # '18':'year2month6',
    # '19':'year2month7',
    # '20':'year2month8',
    # '21':'year2month9',
    # '22':'year2month10',
    # '23':'year2month11',
    # '24':'year2month12',
}
  for month, time_period in time_periods.items():
    CSV_URL = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol=%(stock)s&interval=1min&slice=%(slice)s&apikey=%(key)s&adjusted=false'\
            %{'stock':stock_name,'slice':time_period,'key':API_KEY}
    with requests.Session() as s:
      download = s.get(CSV_URL)
      decoded_content = download.content.decode('utf-8')
      cr = csv.reader(decoded_content.splitlines(), delimiter=',')
      my_list = list(cr)
      for row in my_list:
          write_to_csv('test', row,'a')
get_stock_data('IBM')          

In [155]:
stock = 'AAPL'
mode = 'r'
interval = 'minute'
file_path = stock+'_' + interval +'.csv'
AAPL_minute_data = []
with open(file_path) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
            AAPL_minute_data.append(list(row))
print('done')
print(len(AAPL_minute_data))

done
406974


In [156]:
print(AAPL_minute_data[0])

['12/1/2022 20:00', '147.85', '147.86', '147.82', '147.86', '1137']


In [157]:
daily_opening_candle = []


In [84]:
# DO NOT RUN THIS CELL AND NEXT CELL
reverse_flag = False;

In [85]:
# I SAID DO NOT RUN MF
AAPL_minute_data.reverse()
# can use a flag to check if the file has been reversed
reverse_flag = not reverse_flag
print("Has the list been reversed from its initial state? ", reverse_flag)

Has the list been reversed from its initial state?  True


In [158]:
date_and_time = AAPL_minute_data[0][0].split()
current_date = date_and_time[0]
for i in range(len(AAPL_minute_data)):
  # current_datetime = AAPL_minute_data[i][0]
  date_and_time = AAPL_minute_data[i][0].split()
  end_date = date_and_time[0]
  

  # if there is a change in the current_date, get the data
  if (current_date != end_date):

    # print('If', close_datetime.date(), datetime.strptime( current_datetime, '%m/%d/%Y %H:%M').date())
    # print('Current Date', current_date)
    daily_opening_candle.append(AAPL_minute_data[i-1])
    current_date = end_date
    # print(close_datetime)
  # else:
  #   print('Else', close_datetime.date(), datetime.strptime( current_datetime, '%m/%d/%Y %H:%M').date())
  # counter+=1  

daily_opening_candle.append(AAPL_minute_data[-1])

In [159]:
print(len(daily_opening_candle))
print(daily_opening_candle[0])
print(daily_opening_candle[1])


496
['12/1/2022 4:01', '148.08', '148.08', '147.28', '147.73', '3613']
['11/30/2022 4:01', '143.5', '143.82', '141.5', '141.73', '5617']


In [166]:
for item in daily_opening_candle:
  print(item)

['12/1/2022 4:01', '148.08', '148.08', '147.28', '147.73', '3613']
['11/30/2022 4:01', '143.5', '143.82', '141.5', '141.73', '5617']
['11/29/2022 4:01', '145.04', '145.55', '145.04', '145.48', '4767']
['11/28/2022 4:01', '147.05', '147.64', '146.35', '146.38', '9272']
['11/25/2022 4:01', '151.16', '151.16', '150.56', '150.76', '4920']
['11/23/2022 4:01', '149.75', '149.92', '149.68', '149.92', '8101']
['11/22/2022 4:01', '147.91', '147.91', '147.67', '147.79', '2129']
['11/21/2022 4:01', '150.1', '150.65', '150.1', '150.65', '1784']
['11/18/2022 4:01', '150.17', '150.69', '150.17', '150.5', '2941']
['11/17/2022 4:01', '148.98', '149.26', '148.98', '149.15', '3372']
['11/16/2022 4:01', '150.25', '150.28', '149.75', '150.01', '13170']
['11/15/2022 4:01', '149.51', '149.98', '149.51', '149.81', '3192']
['11/14/2022 4:01', '149.24', '149.24', '148.9', '148.9', '3001']
['11/11/2022 4:01', '148.06', '148.74', '147.86', '148.51', '11535']
['11/10/2022 4:01', '134.87', '135.8', '134.87', '135.

In [161]:
daily_closing_candle = []

In [162]:
current_date = ''
for i in range(len(AAPL_minute_data)):
  # current_datetime = AAPL_minute_data[i][0]
  date_and_time = AAPL_minute_data[i][0].split()
  end_date = date_and_time[0]

  # if there is a change in the current_date, get the data
  if (current_date != end_date):

    # print('If', close_datetime.date(), datetime.strptime( current_datetime, '%m/%d/%Y %H:%M').date())
    # print('Current Date', current_date)
    daily_closing_candle.append(AAPL_minute_data[i])
    current_date = end_date
    # print(close_datetime)
  # else:
  #   print('Else', close_datetime.date(), datetime.strptime( current_datetime, '%m/%d/%Y %H:%M').date())
  # counter+=1  


In [168]:
print(len(daily_closing_candle))
for item in daily_closing_candle:
  print(item)

496
['12/1/2022 20:00', '147.85', '147.86', '147.82', '147.86', '1137']
['11/30/2022 20:00', '148.12', '148.18', '148.11', '148.18', '2404']
['11/29/2022 20:00', '141.29', '141.31', '141.27', '141.31', '3193']
['11/28/2022 20:00', '144.48', '144.5', '144.48', '144.48', '1710']
['11/25/2022 17:00', '148.11', '148.12', '148.11', '148.12', '5082']
['11/23/2022 20:00', '151.1599', '151.16', '151.15', '151.15', '1663']
['11/22/2022 20:00', '150.08', '150.1', '150.05', '150.1', '15893']
['11/21/2022 20:00', '148.15', '148.15', '148.12', '148.12', '849']
['11/18/2022 20:00', '150.85', '150.88', '150.85', '150.88', '773']
['11/17/2022 20:00', '151.03', '151.03', '150.9', '150.9', '1415']
['11/16/2022 20:00', '148.93', '148.93', '148.9', '148.9', '2614']
['11/15/2022 20:00', '149.72', '149.73', '149.69', '149.7', '2981']
['11/14/2022 20:00', '148.71', '148.76', '148.71', '148.76', '990']
['11/11/2022 20:00', '149.45', '149.45', '149.4', '149.44', '1962']
['11/10/2022 20:00', '147.2', '147.2', '

In [169]:
# print(IBM_minute_data[69000:69400])
# Remove ['2021-11-26 09:31:00', ..] to clean data
print(daily_opening_candle[0])
print(daily_closing_candle[0])
print(daily_opening_candle[1])
print(daily_closing_candle[1])
print(daily_opening_candle[-1])
print(daily_closing_candle[-1])
# daily_open_price.pop(160)

['12/1/2022 4:01', '148.08', '148.08', '147.28', '147.73', '3613']
['12/1/2022 20:00', '147.85', '147.86', '147.82', '147.86', '1137']
['11/30/2022 4:01', '143.5', '143.82', '141.5', '141.73', '5617']
['11/30/2022 20:00', '148.12', '148.18', '148.11', '148.18', '2404']
['12/14/2020 4:01', '122.2', '122.3', '122.2', '122.26', '5379']
['12/14/2020 20:00', '121.89', '121.89', '121.89', '121.89', '895']


In [ ]:
#  Clean minute data to remove headers
# for i in range(len(IBM_minute_data)):
#   if IBM_minute_data[i][0] == '{':
#     # print(IBM_minute_data[i])
#     IBM_minute_data.pop(i)
#     # print(daily_open_price[i][0], "||",daily_close_price[i][0])



In [133]:
PRINCIPAL = 1000
returns = []

In [134]:
def close_trade(gain_or_loss):
  global PRINCIPAL
  returns.append(gain_or_loss)
  PRINCIPAL += gain_or_loss


In [135]:
# Principal
# Current_principal = Principal - buying_price
# Once stock_price meets desired profit(gain)
# Exit trade => Current_principal = current_principal + buying_price + Gain
########################### Go bull ##################

# Buy
# Wait until you make desired profit or hit the SL
# Exit
def go_bull(profit_percent,buy_price,time_series_data):
  # print(time_series_data[0],'Timeseries dataaaaaaaaaaaaaaa yoooooooo')
  target = buy_price + (buy_price * profit_percent)
  stop_loss = profit_percent/4
  bottom_line = buy_price - (buy_price*stop_loss)
  print(buy_price,target,bottom_line)
  for row in time_series_data:
    # Check for stop loss first
    #  Comparing with open and low 
    if(max(float(row[1])  , float(row[3]))  <= bottom_line):
      print("SL Hit")
      print(buy_price,bottom_line,'max(float(row[1]), float(row[3]))', max(float(row[1]), float(row[3])))
      close_trade(max(float(row[1])  , float(row[3])) - buy_price)
      break
    # Comparing with open and high 
    if(float(row[1]) , float(row[2])  >= target):
      print("Profit Hit")
      print(buy_price,target,'max(float(row[1]), float(row[2]))', max(float(row[1]), float(row[2])))
      close_trade(max(float(row[1])  , float(row[2])) - buy_price)
      break
  close_trade(buy_price - float(time_series_data[-1][4]))   


In [136]:
# Principal
# Current_principal = Principal - buying_price
# Once stock_price meets desired profit(gain)
# Exit trade => Current_principal = current_principal + buying_price + Gain
########################### Go bear ##################

# Buy
# Wait until you make desired profit or hit the SL
# Exit

def go_bear(profit_percent, sell_price, time_series_data):
  # print(time_series_data[0],'Timeseries dataaaaaaaaaaaaaaa')
  target = sell_price - (sell_price * profit_percent)
  stop_loss = profit_percent/4
  top_line = sell_price + (sell_price*stop_loss)
  print(sell_price,target,top_line)

  # for row in time_series_data:
  #   print(row)


  for row in time_series_data:
    # Check for stop loss first
    #  Comparing with open and high
    if(min(float(row[1]), float(row[2]))  >= top_line):
      print("SL Hit")
      print(sell_price,top_line,'min(float(row[1]), float(row[2]))', min(float(row[1]), float(row[2])))
      close_trade(sell_price - min(float(row[1])  , float(row[2])))
      break
    # Comparing with open and low 
    if(min(float(row[1]),float( row[3]))  <= target):
      print("Profit Hit")
      print(sell_price, target,'min(float(row[1]), float(row[3]))', min(float(row[1]), float(row[3])))
      close_trade(sell_price - min(float(row[1])  , float(row[3])))
      break
  # Neither conditions hit. Close at the end of the trading day 
  close_trade(sell_price - float(time_series_data[-1][4]))   

In [137]:
def get_minute_data(date):

  minute_data = []
  print(date)

  # Column header should not be considered hence len - 1
  for i in range(1, len(AAPL_minute_data) - 1):
      current_date = AAPL_minute_data[i][0].split()[0]
      if (current_date == date):
        minute_data.append(AAPL_minute_data[i])
        next_date = AAPL_minute_data[i+1][0].split()[0]
        if (current_date != next_date):
          break
  minute_data   
  return minute_data  
# print(get_minute_data(daily_closing_candle[0][0].split()[0]))  
# print(AAPL_minute_data[1][0])  

12/1/2022
[['12/1/2022 19:58', '147.88', '147.88', '147.88', '147.88', '453'], ['12/1/2022 19:56', '147.9', '147.9', '147.9', '147.9', '204'], ['12/1/2022 19:55', '147.9', '147.9', '147.9', '147.9', '348'], ['12/1/2022 19:53', '147.9', '147.9', '147.85', '147.9', '2004'], ['12/1/2022 19:52', '147.86', '147.86', '147.86', '147.86', '277'], ['12/1/2022 19:51', '147.91', '147.91', '147.86', '147.88', '3220'], ['12/1/2022 19:50', '147.91', '147.91', '147.86', '147.86', '1560'], ['12/1/2022 19:49', '147.93', '147.95', '147.91', '147.91', '1401'], ['12/1/2022 19:48', '147.93', '147.93', '147.93', '147.93', '592'], ['12/1/2022 19:47', '147.93', '147.93', '147.93', '147.93', '293'], ['12/1/2022 19:45', '147.93', '147.93', '147.93', '147.93', '1015'], ['12/1/2022 19:43', '147.93', '147.93', '147.93', '147.93', '309'], ['12/1/2022 19:42', '147.97', '147.98', '147.97', '147.98', '2039'], ['12/1/2022 19:41', '147.93', '148.1', '147.93', '147.95', '2941'], ['12/1/2022 19:40', '147.95', '147.95', '1

In [171]:
# print(daily_opening_candle[0])
# print(daily_closing_candle[0])

# daily_open_price.reverse()
# daily_close_price.reverse()
for item in daily_opening_candle:
  print(item)

['12/1/2022 4:01', '148.08', '148.08', '147.28', '147.73', '3613']
['11/30/2022 4:01', '143.5', '143.82', '141.5', '141.73', '5617']
['11/29/2022 4:01', '145.04', '145.55', '145.04', '145.48', '4767']
['11/28/2022 4:01', '147.05', '147.64', '146.35', '146.38', '9272']
['11/25/2022 4:01', '151.16', '151.16', '150.56', '150.76', '4920']
['11/23/2022 4:01', '149.75', '149.92', '149.68', '149.92', '8101']
['11/22/2022 4:01', '147.91', '147.91', '147.67', '147.79', '2129']
['11/21/2022 4:01', '150.1', '150.65', '150.1', '150.65', '1784']
['11/18/2022 4:01', '150.17', '150.69', '150.17', '150.5', '2941']
['11/17/2022 4:01', '148.98', '149.26', '148.98', '149.15', '3372']
['11/16/2022 4:01', '150.25', '150.28', '149.75', '150.01', '13170']
['11/15/2022 4:01', '149.51', '149.98', '149.51', '149.81', '3192']
['11/14/2022 4:01', '149.24', '149.24', '148.9', '148.9', '3001']
['11/11/2022 4:01', '148.06', '148.74', '147.86', '148.51', '11535']
['11/10/2022 4:01', '134.87', '135.8', '134.87', '135.

In [145]:
PRINCIPAL = 1000
for i in range(len(daily_opening_candle)-1):
  
  
  print(i,i+1)
  prev_stock_price = float(daily_closing_candle[i][1])
  stock_price = float(daily_opening_candle[i+1][1])
  bullish_condition = stock_price > prev_stock_price * 1.01
  bearish_condition = stock_price < prev_stock_price * 0.99
  # print(stock_price,'|',prev_stock_price)
  # print(bullish_condition)
  # print(bearish_condition)

  if(bullish_condition):
    # go_bull
    time_series = get_minute_data(daily_opening_candle[0][0].split()[0])
    print("Going bull",time_series)
    go_bull(0.01, stock_price,time_series)
    None
  elif(bearish_condition):
    print("Going bear")
    time_series = get_minute_data(daily_opening_candle[0][0].split()[0])
    go_bear(0.01, stock_price,time_series)

    # go bear

    # Exec Trade


0 1
1 2
2 3
3 4
4 5
5 6
6 7
12/14/2020
Going bull [['12/14/2020 20:00', '121.89', '121.89', '121.89', '121.89', '895'], ['12/14/2020 19:59', '121.84', '121.84', '121.84', '121.84', '679'], ['12/14/2020 19:58', '121.89', '121.89', '121.84', '121.89', '1800'], ['12/14/2020 19:57', '121.84', '121.89', '121.84', '121.89', '877'], ['12/14/2020 19:56', '121.89', '121.89', '121.89', '121.89', '245'], ['12/14/2020 19:55', '121.89', '121.89', '121.86', '121.89', '938'], ['12/14/2020 19:54', '121.89', '121.89', '121.89', '121.89', '549'], ['12/14/2020 19:53', '121.9', '121.9', '121.9', '121.9', '404'], ['12/14/2020 19:52', '121.83', '121.85', '121.8', '121.82', '4253'], ['12/14/2020 19:51', '121.91', '121.91', '121.91', '121.91', '378'], ['12/14/2020 19:50', '121.82', '121.84', '121.82', '121.84', '448'], ['12/14/2020 19:49', '121.85', '121.85', '121.81', '121.81', '720'], ['12/14/2020 19:48', '121.85', '121.85', '121.85', '121.85', '223'], ['12/14/2020 19:47', '121.83', '121.85', '121.81', '121

In [146]:
print(PRINCIPAL)
print(returns)

1718.5799999999992
[-4.3799999999999955, -4.22999999999999, -2.8400000000000034, -2.6899999999999977, 0.8299999999999841, -0.6799999999999784, -3.280000000000001, 3.430000000000007, -1.8700000000000045, 2.0200000000000102, 0.030000000000001137, 0.18000000000000682, -2.289999999999992, 2.4399999999999977, 1.0999999999999943, 1.25, -13.009999999999991, -12.859999999999985, -7.829999999999984, -7.679999999999978, 8.840000000000003, -8.689999999999998, -9.879999999999995, -9.72999999999999, -9.870000000000005, -9.719999999999999, 2.960000000000008, -2.8100000000000023, -2.8400000000000034, 2.990000000000009, -5.710000000000008, 5.860000000000014, -7.1200000000000045, 7.27000000000001, -4.189999999999998, -4.039999999999992, 1.0600000000000023, 1.210000000000008, -3.180000000000007, 3.3300000000000125, 0.020000000000010232, 0.17000000000001592, -5.090000000000003, -4.939999999999998, -5.389999999999986, -5.239999999999981, -4.889999999999986, -4.739999999999981, -7.659999999999997, -7.50999

In [ ]:
from datetime import datetime
datetime_str = '2022-10-10 09:30:00'
date_str = '2022-10-10'
now = datetime.strptime(date_str, '%Y-%m-%d')
timestamp = datetime.timestamp(now)
print("timestamp =", timestamp)


timestamp = 1665360000.0
